# Import Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import matplotlib as mpl
import matplotlib.pyplot as plt


tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

# Setting the random seed

In [ ]:
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Dataset configuration 


In [ ]:
#additional component to perform the split
!pip install split-folders 
import splitfolders

In [ ]:
splitfolders.ratio('/kaggle/input/images/training', output='outputDataset', seed = seed, ratio=(0.85, 0.1, 0.05))
#in this case we set a path compatible to the kaggle environment, if the program in run on colab we also need to perform an unzip the first time.
#85% testing
#10 validation
#5% testing


In [ ]:
# Dataset folders are set here
dataset_dir = 'outputDataset'
training_dir = os.path.join(dataset_dir, 'train')
validation_dir = os.path.join(dataset_dir, 'val')
test_dir = os.path.join(dataset_dir, 'test')

# Model Parameters

In [ ]:
labels = ['Apple',              # 0
          'Blueberry',          # 1
          "Cherry",             # 2
          "Corn",               # 3
          "Grape",              # 4
          "Orange",             # 5
          "Peach",              # 6
          "Pepper",             # 7
          "Potato",             # 8
          "Raspberry",          # 9
          "Soybean",            # 10
          "Squash",             # 11
          "Strawberry",         # 12
          "Tomato"]             # 13

number_of_classes = 14

In [ ]:
input_shape = (256, 256, 3) #3 channel of RGB #256x256 the size 
image_width = 256
image_height = 256

epochs = 250 
patience_epochs = 30 
batch_size = 128 

# Data Augmentation

In [ ]:
#Here we set the generators for the data augmentation process
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(rotation_range=145,
                                        height_shift_range=80, 
                                        width_shift_range=80,
                                        zoom_range=0.5,
                                        horizontal_flip=True,
                                        vertical_flip=True, 
                                        fill_mode='constant', #if necessary we fill with black pixels
                                        cval=0.0,
                                        brightness_range=[0.5,1.5], #1.0 denotes the image at the original brightness level
                                        shear_range=0.3,    
                                        rescale=1/255.) 
                                        
train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=labels, 
                                               class_mode='categorical',
                                               batch_size=batch_size, 
                                               shuffle=True,
                                               seed=seed)

valid_data_gen = ImageDataGenerator(rescale=1/255.)
valid_gen = train_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=labels, 
                                               class_mode='categorical',
                                               batch_size=batch_size,
                                               shuffle=False,
                                               seed=seed)

test_data_gen = ImageDataGenerator(rescale=1/255.)
test_gen = train_data_gen.flow_from_directory(directory=test_dir,
                                              target_size=(256,256),
                                              color_mode='rgb',
                                              classes=labels, 
                                              class_mode='categorical',
                                              batch_size=batch_size,
                                              shuffle=False,
                                              seed=seed)

# Model Configuration

In [ ]:
def build_model(input_shape):

    #Build the neural network layer by layer
    #input  shape 256x256
    input_layer = tfkl.Input(shape=input_shape, name='Input')


    #block n.1 (conv + batch + maxPooling)
    conv1 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same', #no reduction of the dimensions of the image
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(input_layer)  
    
    conv1 = tfkl.BatchNormalization(axis=-1, scale=False, name='norm1')(conv1)
    conv1 = tfkl.Activation('relu')(conv1)
    
    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv1)

    #block n.2 (conv + batch + maxPooling)
    conv2 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1) 
    
    conv2 = tfkl.BatchNormalization(axis=-1, scale=False, name='norm2')(conv2)
    conv2 = tfkl.Activation('relu')(conv2)
    
    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv2)

    #block n.3 (conv + batch + maxPooling)
    conv3 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2) 
    
    conv3 = tfkl.BatchNormalization(axis=-1, scale=False, name='norm3')(conv3)
    conv3 = tfkl.Activation('relu')(conv3)
    
    
    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv3)

    #block n.4 (conv + batch + maxPooling)
    conv4 = tfkl.Conv2D(
        filters=256,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool3)  
    
    conv4 = tfkl.BatchNormalization(axis=-1, scale=False, name='norm4')(conv4)
    conv4 = tfkl.Activation('relu')(conv4)
    
    pool4 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv4)

    #block n.5 (conv + batch + maxPooling)
    conv5 = tfkl.Conv2D(
        filters=512,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool4) 
    
    conv5 = tfkl.BatchNormalization(axis=-1, scale=False, name='norm5')(conv5)
    conv5 = tfkl.Activation('relu')(conv5)
    
    pool5 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv5)
    
    #block n.6 (conv + batch + maxPooling)
    conv6 = tfkl.Conv2D(
        filters=1024,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool5)
    
    conv6 = tfkl.BatchNormalization(axis=-1, scale=False, name='norm6')(conv6)
    conv6 = tfkl.Activation('relu')(conv6)
    
    pool6 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv6)

    
    glob_pooling = tfkl.GlobalAveragePooling2D(name='GloablPooling')(pool6)
    
    #Fully connected network
    classifier_layer1 = tfkl.Dense(units=1024, name='Classifier1', kernel_initializer=tfk.initializers.GlorotUniform(seed), kernel_regularizer=tf.keras.regularizers.l2(2e-6), activation='relu')(glob_pooling)
    classifier_layer2 = tfkl.Dense(units=512, name='Classifier2', kernel_initializer=tfk.initializers.GlorotUniform(seed), kernel_regularizer=tf.keras.regularizers.l2(2e-6), activation='relu')(classifier_layer1)
    classifier_layer2 = tfkl.Dropout(0.6, seed=seed)(classifier_layer2)
    classifier_layer3 = tfkl.Dense(units=256, name='Classifier3', kernel_initializer=tfk.initializers.GlorotUniform(seed), kernel_regularizer=tf.keras.regularizers.l2(2e-6), activation='relu')(classifier_layer2)
    classifier_layer3 = tfkl.Dropout(0.5, seed=seed)(classifier_layer3)
    classifier_layer4 = tfkl.Dense(units=64, name='Classifier4', kernel_initializer=tfk.initializers.GlorotUniform(seed), kernel_regularizer=tf.keras.regularizers.l2(2e-6), activation='relu')(classifier_layer3)
    
    #Output layer characterized by the softmax activation function
    output_layer = tfkl.Dense(units=number_of_classes, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), kernel_regularizer=tf.keras.regularizers.l2(2e-6), name='Output')(classifier_layer4)
    
    #Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    
    return model

In [ ]:
model = build_model(input_shape) 
model.summary()

In [ ]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('data_augmentation_experiments')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, 
                                                     save_best_only=True, #saves only the best evaluating the val_accuracy
                                                     monitor='val_accuracy',
                                                     mode='max')  
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)

  # Early Stopping
  # -------------- #patience epochs settato da noi 
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience_epochs, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

# Training

In [ ]:
tf.get_logger().setLevel('WARNING') #  if you want to suppress only INFOs
tf.get_logger().setLevel('ERROR') #  if you want to suppress both WARNINGs and INFOs

# Create folders and callbacks and fit
aug_callbacks = create_folders_and_callbacks(model_name='CNN')

# Train the model
history = model.fit(
    x = train_gen,
    epochs = epochs,
    validation_data = valid_gen,
    callbacks = aug_callbacks,
).history

model.save("ModelDataAug")

# Testing

In [ ]:
model_aug = tfk.models.load_model('ModelDataAug')
model_metrics = model_aug.evaluate(test_gen, return_dict = True)
print(model_metrics)

#Graphs

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history['loss'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
plt.ylim(0, 0.1)
plt.title('val_loss and train_loss')
plt.legend(loc='upper right')
plt.grid(alpha=.3)
plt.show()